In [2]:
import pandas as pd
import xgboost as xgb
import pickle 
model = pickle.load(open('../models/model_xgb.pck', 'rb'))


In [6]:
from datetime import datetime

df = pd.read_hdf('../data/processed/train/aggr_train_manual_feat.hdf')
threshold = datetime(2015, 9, 1)
df_valid['date'] = pd.to_datetime(df_valid['year'].astype(str) + '-' + df_valid['month'].astype(str) + '-1')
df_valid = df[df['date'] >= threshold]

In [7]:

def X(df):
    df['day_of_year'] = df['date'].dt.dayofyear
    date_cols = [
        'month', 'year',
        'fridays', 'day_of_year'
    ]
    one_hot_cols = [
        'shop_id', 'mall', 'city',
        'item_id', 'item_category_id', 'first_big_category', 'last_big_category',
    ]
    df_X = df[
        [
            col for col in df.columns
            if col.startswith('count')
        ]
        + date_cols
        ]

    # df_X = pd.get_dummies(df_X, columns=one_hot_cols)
    # df_X = pd.get_dummies(df_X, columns=one_hot_cols + date_cols)
    return df_X


def XY(df):
    df_X = X(df)
    df_y = df['item_cnt_day']
    return df_X, df_y

In [8]:

df_X_valid, df_y_valid = XY(df_valid)

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
def get_preds_trained_on_all(model, df_X):
    return model.predict(xgb.DMatrix(df_X)).clip(0, 20)


def previous_value_preds(df_X):
    df_X = df_X.reindex(columns=df_X_train.columns, fill_value=0)
    return df_X['count_aggr_1_month_ago_shop_id_item_id'].clip(0, 20)


def validate(model, df_y_valid, df_X_valid):
    df_y_valid = df_y_valid.copy().clip(0, 20)
    df_y_valid_preds = get_preds_trained_on_all(model, df_X_valid)
    # df_y_valid_preds = previous_value_preds(df_X_valid)
    print('r2 score: ', r2_score(df_y_valid, df_y_valid_preds))
    print('RMSE: ', sqrt(mean_squared_error(df_y_valid, df_y_valid_preds)))


In [15]:
validate(model, df_y_valid, df_X_valid)

r2 score:  0.35349865676600745
RMSE:  0.8868060725164463


In [21]:

df_test = pd.read_hdf('../data/processed/test/aggr_test_manual_feat.hdf')
df_test_X = X(df_test)
df_test['item_cnt_month'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'item_cnt_month']].to_csv('../data/submissions/submission_trained_on_year_xgb.csv', index=False)

linreg_test = pd.read_csv('../data/submissions/if_sells_submission_linreg.csv')
print('linreg RMSE: ', sqrt(mean_squared_error(linreg_test['item_cnt_month'], df_test['item_cnt_month'])))
rf_test = pd.read_csv('../data/submissions/if_sells_submission_rf.csv')
print('rf RMSE: ', sqrt(mean_squared_error(rf_test['item_cnt_month'], df_test['item_cnt_month'])))

linreg RMSE:  1.2656266149507325
rf RMSE:  2.0776610156837596
